# Mudança de temperatura global, um estudo de caso.

<img src="https://i.imgur.com/VKiO62H.png" width="400" height="400">

> ### Aplicação do método dos minímos quadrados em uma base de dados sobre a mudança de temperatura global, dados fornecidos pela NASA.

> Dados sintetizados em decadas para minimizar o erro da regressão




## Montando drive e lendo arquivo com a base de dados


In [ ]:
# Imports necessários
import numpy as np
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

dados_iniciais = pd.read_csv('/content/drive/MyDrive/2023.1/Calculo numérico/data/dados_decada.csv') # TODO: FORMATAR 15 CASAS DECIMAIS
dados_iniciais['Temperatura_media'] = dados_iniciais['Temperatura_media'].str.replace(',', '.').astype(float)
dados_iniciais.head()

Mounted at /content/drive


,Ano(x),Temperatura_media
0,1880,13.477273
1,1890,13.494000
2,1900,13.354000
3,1910,13.391000
4,1920,13.473000


# Classe para resolução de sistemas de equações lineares
> Eliminação de Gauss

In [ ]:
def print_matriz(matrix):
    num_rows = len(matrix[0])
    num_columns = len(matrix[0])

    # Calcular a largura da célula com base no maior valor da matriz
    max_value = max(max(row) for row in matrix)
    cell_width = len(f"{max_value:.2f}") + 2

    # Calcular a largura total
    total_width = num_columns * cell_width + 1

    # Imprimir a linha superior da matriz
    print("┌" + "─" * total_width + "┐")

    # Imprimir os elementos da matriz
    for row in matrix:
        print("│", end="")  # Imprime a borda esquerda da matriz
        for element in row:
            print(f" {element:.2f} ".rjust(cell_width), end="")
        print("│")  # Imprime a borda direita da matriz e pula para a próxima linha

    # Imprimir a linha inferior da matriz
    print("└" + "─" * total_width + "┘")


'''
    Temos o seguinte sistema linear como exemplo:
        |3x1 + 2x2 + 4x3 = 1
        | x1 +  x2 + 2x3 = 2
        |4x1 + 3x2 - 2x3 = 3
    Os métodos implementados serão validados a partir do exemplo acima.
'''

class SistEqLineares:

    def __init__(self, A, b):
      # Definindo matriz dos coeficientes A
      self.A = np.array(A, dtype="float64")
      # Definindo vetor dos termos independentes "b"
      self.b = np.array(b, dtype="float64")
      self.A_B = np.array([])

    def monta_matriz_aumentada(A, b):
        '''
            A função zip combina os elementos da matriz A com o vetor B,
            onde o primeiro elemento é uma linha da matriz A e o segundo elemento é
            o valor correspondente de b, então é utilizado list comprehension para
            iterar sobre as tuplas geradas pelo zip e criar a matriz aumentada A|B

            alternativa usando list comprehesion:
            return [row + [b_val] for row, b_val in zip(A, b)]

        '''

        return np.column_stack((A, b))

    # Método direto da eliminação Gaussiana
    @classmethod
    def eliminacao_gauss(cls,self, A, b):
        # Definindo a matriz aumentada [A|B]
        self.A_B = cls.monta_matriz_aumentada(A, b)
        nl, nc = len(A), len(A[0])

        for i in range(0, nl):
            print("K = ", i+1)  # Controlando o numéro de iterações
            print_matriz(self.A_B)   # Printando matriz de maneira formatada

            pivo = self.A_B[i, i]

            for j in range(i + 1, nl):                  # Percorre as linhas abaixo da linha atual
                mult_linha = self.A_B[j, i] / pivo           # ML = [A|B]_ij / pivo
                self.A_B[j, :] -= mult_linha * self.A_B[i, :]     # L_i <- L_i - ML_i * L_pivo

        # Agora, a resolução do sistema é trivial, basta aplicar a retrosubstituição para achar a, b e c

        #print_matriz(A)
        #print_matriz(b)

    # Método direto da fatoração L.U.
    def fatoracao_lu(cls, A, b):
        return 0

    # Método iterativo de Gauss-Jacobi
    def gauss_jacobi(cls, A, b):
        return 0

    # Método iterativo de Gauss-Seidel
    def gauss_seidel(cls, A, b):
        return 0

## Funções auxiliares para o método

In [ ]:
def tabelamento_inicial(dados):

  # Tabelamento geral do método dos minímos quadrados
  dados['X'] = None
  dados['Y'] = None
  dados['x*y'] = None
  dados['x^2'] = None
  dados['x^3'] = None
  dados['x^4'] = None
  dados['(x^2)*Y'] = None
  dados['S_Q_REG'] = None
  #dados['S_Q_RES'] = None
  dados['S_Q_TOTAL'] = None
  dados['g(x)'] = None

  return dados

# Definindo somatório
def somatorio(vet):
    somatorio = 0
    for i in range(len(vet)):
        somatorio += vet[i]
    return somatorio

# Retorna x * y
def xy(dados):
  return dados['X'] * dados['Y']

def x_quadrado(x):
   return x * x

def x_quadrado_y(dados):
  return (dados['X'] * dados['X']) * dados['Y']


'''
'''

# Soma dos quadrados da regressão
def s_q_regressao(funcao_g, y_medio, dados_regressao):
  # Aplicando funcao g(x) a cada ponto
  valores_previstos = dados_regressao['X'].apply(lambda x: funcao_g(a, b, x))

  # Calculando erro médio
  erros = valores_previstos.apply(lambda x: x - y_medio)

  # Preenchendo tabelamento
  dados_regressao['S_Q_REG'] = erros * erros

  return somatorio(erros * erros)

# Soma dos quadrados residuais
def s_q_residual(funcao_g, dados_regressao):
  # Aplicando funcao g(x) a cada ponto
  valores_previstos = dados_regressao['Ano(x)'].apply(lambda x: funcao_g(a, b, x))

  # Calculando erro residual
  erros = dados_regressao['Temperatura_media'] - valores_previstos

  # Preenchendo tabelamento
  dados_regressao['S_Q_RES'] = erros * erros

  return somatorio(erros * erros)

# Soma dos quadrados Total
def s_q_total(dados_regressao, y_medio):

  dados = dados_regressao['Temperatura_media'] - y_medio

  # Preenchendo tabelamento
  dados_regressao['S_Q_TOTAL'] = dados * dados

  return somatorio(dados * dados)

# Calculo do R quadrado
def r_quadrado(s_q_res, s_q_total):
  return 1 - (s_q_res / s_q_total)

def printa_resultado(dados, y_medio, funcao):
  print(f"Soma dos Quadrados da Regressão (SSR): {s_q_regressao(funcao, y_medio, dados)}")
  print(f"Soma dos Quadrados dos Erros (SSE): {s_q_residual(funcao, dados_nasa)}")
  print(f"Soma dos Quadrados Total (SST): {s_q_total(dados_nasa, y_medio)}")
  print(f"R2: {r_quadrado(s_q_residual(funcao, dados_nasa), s_q_total(dados, y_medio))}")

## Montando tabelamento inicial

1. **Recortando dados da nasa**
  > Aplicada transformação nos dados de modo a criar apenas uma tabela para a base de dados da NASA, pois o estudo da equipe se baseou nesses dados.
2. **Adicionando colunas que precisam ser geradas para o calculo da regressão**
> x * y, x ^ 2, S_Q_REG_, S_Q_RES, S_Q_TOTAL




In [ ]:
dados_nasa = dados_iniciais
dados_nasa = tabelamento_inicial(dados_nasa)
dados_nasa

,Ano(x),Temperatura_media,X,Y,x*y,x^2,x^3,x^4,(x^2)*Y,S_Q_REG,S_Q_TOTAL,g(x)
0,1880,13.477273,None,None,None,None,None,None,None,None,None,None
1,1890,13.494000,None,None,None,None,None,None,None,None,None,None
2,1900,13.354000,None,None,None,None,None,None,None,None,None,None
3,1910,13.391000,None,None,None,None,None,None,None,None,None,None
4,1920,13.473000,None,None,None,None,None,None,None,None,None,None
5,1930,13.613000,None,None,None,None,None,None,None,None,None,None
6,1940,13.718000,None,None,None,None,None,None,None,None,None,None
7,1950,13.667000,None,None,None,None,None,None,None,None,None,None
8,1960,13.675000,None,None,None,None,None,None,None,None,None,None
9,1970,13.759000,None,None,None,None,None,None,None,None,None,None


# Caso linear
> A função que representa o caso linear é a função da reta
$$
   y = a*x+b
$$


In [ ]:
# Determinando função da reta
def linear(a, b, x):
    return (a * x) + b

def linear2(x, a, b):
    return (a * x) + b

## Montando colunas x, y, x * y e x^2

In [ ]:
# Criando dataframe para o caso linear
dados_nasa_linear = tabelamento_inicial(dados_nasa)

# Preenchendo X e Y do caso linear
dados_nasa_linear['X'] = dados_nasa_linear['Ano(x)']
dados_nasa_linear['Y'] = dados_nasa_linear['Temperatura_media']

# Aplicando x * y e x^2 para preencher valores no tabelamento
dados_nasa_linear['x*y'] = dados_nasa_linear[['X', 'Y']].apply(xy, axis=1)
dados_nasa_linear['x^2'] = dados_nasa_linear['X'].apply(x_quadrado)
dados_nasa_linear.head()

,Ano(x),Temperatura_media,X,Y,x*y,x^2,x^3,x^4,(x^2)*Y,S_Q_REG,S_Q_TOTAL,g(x)
0,1880,13.477273,1880,13.477273,25337.272727,3534400,None,None,None,None,None,None
1,1890,13.494000,1890,13.494000,25503.660000,3572100,None,None,None,None,None,None
2,1900,13.354000,1900,13.354000,25372.600000,3610000,None,None,None,None,None,None
3,1910,13.391000,1910,13.391000,25576.810000,3648100,None,None,None,None,None,None
4,1920,13.473000,1920,13.473000,25868.160000,3686400,None,None,None,None,None,None


## Calculo dos coeficientes a e b do caso linear
<img src="https://i.imgur.com/nLoQLOJ.png" width="300">
<img src="https://i.imgur.com/oZvQ54X.png" width="300" height="91">


In [ ]:
# Determinando valor de n baseado no numero de linhas do dataframe
n = dados_nasa_linear.shape[0]

# Calculando somatorio dos valores tabelados para aplicar na equação a e b
somatorio_x = somatorio(dados_nasa_linear['Ano(x)'])
somatorio_y = somatorio(dados_nasa_linear['Temperatura_media'])
somatorio_xy = somatorio(dados_nasa_linear['x*y'])
somatorio_x_quadrado = somatorio(dados_nasa_linear['x^2'])
somatorio_x_ao_quadrado = somatorio_x * somatorio_x

# Calculando coeficiente a
a_linear = ((n * somatorio_xy) - (somatorio_x * somatorio_y)) / ((n * somatorio_x_quadrado) - (somatorio_x_ao_quadrado))
# Calculando coeficiente b
b_linear = ((somatorio_x * somatorio_xy) - (somatorio_y * somatorio_x_quadrado)) / (somatorio_x_ao_quadrado - (n * somatorio_x_quadrado))

print(f'a = {a_linear}\nb = {b_linear}')

a = 0.00830860389610309
b = -2.3950927489189873


## Tabelamento final e resultados do caso linear

In [ ]:
# Cálculo de y médio
y_medio_linear = somatorio(dados_nasa_linear['Y']) / n

dados_nasa_linear['S_Q_REG']   = dados_nasa_linear['X'].apply(lambda x: (((a_linear * x) + b_linear) - y_medio_linear)**2)
dados_nasa_linear['S_Q_TOTAL'] = dados_nasa_linear['Y'].apply(lambda y: (y_medio_linear - y)**2)

SQREG   = somatorio(dados_nasa_linear['S_Q_REG'])
SQTOTAL = somatorio(dados_nasa_linear['S_Q_TOTAL'])


# Cálculo de y médio
print(f"Soma dos Quadrados da Regressão (SSR): { SQREG }")
print(f"Soma dos Quadrados Total (SST): { SQTOTAL }")
print(f"R2: { SQREG / SQTOTAL }")
dados_nasa_linear.head()


Soma dos Quadrados da Regressão (SSR): 1.9329211636655068
Soma dos Quadrados Total (SST): 2.269525584573019
R2: 0.8516851172793278


,Ano(x),Temperatura_media,X,Y,x*y,x^2,x^3,x^4,(x^2)*Y,S_Q_REG,S_Q_TOTAL,g(x)
0,1880,13.477273,1880,13.477273,25337.272727,3534400,None,None,None,0.338261,0.108512,None
1,1890,13.494000,1890,13.494000,25503.660000,3572100,None,None,None,0.248518,0.097772,None
2,1900,13.354000,1900,13.354000,25372.600000,3610000,None,None,None,0.172582,0.204924,None
3,1910,13.391000,1910,13.391000,25576.810000,3648100,None,None,None,0.110453,0.172794,None
4,1920,13.473000,1920,13.473000,25868.160000,3686400,None,None,None,0.062130,0.111346,None


### Aplicando regressão nos pontos Xi para calcular previsões do caso linear

In [ ]:
# TODO APLICAR REGRESSAO NOS PONTOS X
dados_nasa_linear['g(x)'] = dados_nasa_linear['X'].apply(linear2, a=a_linear, b=b_linear)
dados_nasa_linear.head()
dados_nasa_linear.to_csv("dados_nasa_linear.csv")

# Caso logaritmo
> A função que representa o caso logaritmo é a função:
$$
   y = a*ln(x)+b
$$


In [ ]:
# Determinando função logaritmica
def logaritmo(a, b, x):
  return (a * np.log(x)) + b

## Tabelamento inicial e colunas x, y, x * y e x^2

In [ ]:
# Criando dataframe para o caso linear
dados_nasa_logaritmo = tabelamento_inicial(dados_nasa)

# Preenchendo X(logaritmo natural do ponto) e Y do caso linear
dados_nasa_logaritmo['X'] = dados_nasa_logaritmo['Ano(x)'].apply(np.log)
dados_nasa_logaritmo['Y'] = dados_nasa_logaritmo['Temperatura_media']

# Aplicando x * y e x^2 para preencher valores no tabelamento
dados_nasa_logaritmo['x*y'] = dados_nasa_logaritmo[['X', 'Y']].apply(xy, axis=1)
dados_nasa_logaritmo['x^2'] = dados_nasa_logaritmo['X'].apply(x_quadrado)
dados_nasa_logaritmo.head()

## Calculo dos coeficientes a e b do caso logaritmo
<img src="https://i.imgur.com/nLoQLOJ.png" width="300">
<img src="https://i.imgur.com/oZvQ54X.png" width="300" height="91">


In [ ]:
# Determinando valor de n baseado no numero de linhas do dataframe
n = dados_nasa_logaritmo.shape[0]

# Calculando somatorio dos valores tabelados para aplicar na equação a e b
somatorio_x = somatorio(dados_nasa_logaritmo['X'])
somatorio_y = somatorio(dados_nasa_logaritmo['Y'])
somatorio_xy = somatorio(dados_nasa_logaritmo['x*y'])
somatorio_x_quadrado = somatorio(dados_nasa_logaritmo['x^2'])
somatorio_x_ao_quadrado = somatorio_x * somatorio_x

# Calculando coeficiente a
a_logaritmo = ((n * somatorio_xy) - (somatorio_x * somatorio_y)) / ((n * somatorio_x_quadrado) - (somatorio_x_ao_quadrado))
# Calculando coeficiente b
b_logaritmo = ((somatorio_x * somatorio_xy) - (somatorio_y * somatorio_x_quadrado)) / (somatorio_x_ao_quadrado - (n * somatorio_x_quadrado))

print(f'a = {a_logaritmo}\nb = {b_logaritmo}')

## Tabelamento final e resultados do caso logaritmo

In [ ]:
# Cálculo de y médio
y_medio_logaritmo = somatorio(dados_nasa_logaritmo['Y']) / n

dados_nasa_logaritmo['S_Q_REG']   = dados_nasa_logaritmo['X'].apply(lambda x: (((a_logaritmo * x) + b_logaritmo) - y_medio_logaritmo)**2)
dados_nasa_logaritmo['S_Q_TOTAL'] = dados_nasa_logaritmo['Y'].apply(lambda y: (y_medio_logaritmo - y)**2)

SQREG   = somatorio(dados_nasa_logaritmo['S_Q_REG'])
SQTOTAL = somatorio(dados_nasa_logaritmo['S_Q_TOTAL'])

dados_nasa_logaritmo.head()

# Cálculo de y médio
print(f"Soma dos Quadrados da Regressão (SSR): { SQREG }")
print(f"Soma dos Quadrados Total (SST): { SQTOTAL }")
print(f"R2: { SQREG / SQTOTAL }")

# Caso exponencial
> A função que representa o caso exponencial é a função:
$$
   ln(y) = a*x+ln(b)
$$


In [ ]:
# Determinando função exponencial
def exponencial(a, b, x):
  return b * (np.exp(a * x))

## Tabelamento inicial e colunas x, y, x * y e x^2

In [ ]:
# Criando dataframe para o caso exponencial
dados_nasa_exponencial = tabelamento_inicial(dados_nasa)

In [ ]:
# Preenchendo X(logaritmo natural do ponto) e Y do caso linear
dados_nasa_exponencial['X'] = dados_nasa_exponencial['Ano(x)']
dados_nasa_exponencial['Y'] = dados_nasa_exponencial['Temperatura_media'].apply(np.log)

# Aplicando x * y e x^2 para preencher valores no tabelamento
dados_nasa_exponencial['x*y'] = dados_nasa_exponencial[['X', 'Y']].apply(xy, axis=1)
dados_nasa_exponencial['x^2'] = dados_nasa_exponencial['X'].apply(x_quadrado)
dados_nasa_exponencial.head()

## Calculo dos coeficientes a e b do caso exponencial
<img src="https://i.imgur.com/nLoQLOJ.png" width="300">
<img src="https://i.imgur.com/oZvQ54X.png" width="300" height="91">


In [ ]:
# Determinando valor de n baseado no numero de linhas do dataframe
n = dados_nasa_exponencial.shape[0]

# Calculando somatorio dos valores tabelados para aplicar na equação a e b
somatorio_x = somatorio(dados_nasa_exponencial['X'])
somatorio_y = somatorio(dados_nasa_exponencial['Y'])
somatorio_xy = somatorio(dados_nasa_exponencial['x*y'])
somatorio_x_quadrado = somatorio(dados_nasa_exponencial['x^2'])
somatorio_x_ao_quadrado = somatorio_x * somatorio_x

# Calculando coeficiente a
a_exponencial = ((n * somatorio_xy) - (somatorio_x * somatorio_y)) / ((n * somatorio_x_quadrado) - (somatorio_x_ao_quadrado))
# Calculando coeficiente b
b_exponencial = ((somatorio_x * somatorio_xy) - (somatorio_y * somatorio_x_quadrado)) / (somatorio_x_ao_quadrado - (n * somatorio_x_quadrado))

print(f'a = {a_exponencial}\nb = {b_exponencial}')

## Tabelamento final e resultados do caso exponencial

In [ ]:
# Cálculo de y médio
y_medio_exponencial = somatorio(dados_nasa_exponencial['Y']) / n
#euler = 0.5772156649
dados_nasa_exponencial['S_Q_REG']   = dados_nasa_exponencial['X'].apply(lambda x: (((a_exponencial * x) + b_exponencial) - y_medio_exponencial)**2)
dados_nasa_exponencial['S_Q_TOTAL'] = dados_nasa_exponencial['Y'].apply(lambda y: (y_medio_exponencial - y)**2)

SQREG   = somatorio(dados_nasa_exponencial['S_Q_REG'])
SQTOTAL = somatorio(dados_nasa_exponencial['S_Q_TOTAL'])


# Cálculo de y médio
print(f"Soma dos Quadrados da Regressão (SSR): { SQREG }")
print(f"Soma dos Quadrados Total (SST): { SQTOTAL }")
print(f"R2: { SQREG / SQTOTAL }")
dados_nasa_exponencial.head()

# Caso polinomial
> A função que representa o caso polinomial é a função:
$$
   y = a*x^2 + b*x + c
$$


## Tabelamento inicial e colunas x, y, x * y e x^2

In [ ]:
# Criando dataframe para o caso exponencial
dados_nasa_polinomial = tabelamento_inicial(dados_nasa)
dados_nasa_polinomial

,Ano(x),Temperatura_media,X,Y,x*y,x^2,x^3,x^4,(x^2)*Y,S_Q_REG,S_Q_TOTAL,g(x)
0,1880,13.477273,None,None,None,None,None,None,None,None,None,None
1,1890,13.494000,None,None,None,None,None,None,None,None,None,None
2,1900,13.354000,None,None,None,None,None,None,None,None,None,None
3,1910,13.391000,None,None,None,None,None,None,None,None,None,None
4,1920,13.473000,None,None,None,None,None,None,None,None,None,None
5,1930,13.613000,None,None,None,None,None,None,None,None,None,None
6,1940,13.718000,None,None,None,None,None,None,None,None,None,None
7,1950,13.667000,None,None,None,None,None,None,None,None,None,None
8,1960,13.675000,None,None,None,None,None,None,None,None,None,None
9,1970,13.759000,None,None,None,None,None,None,None,None,None,None


In [ ]:
# Preenchendo X(logaritmo natural do ponto) e Y do caso linear
dados_nasa_polinomial['X'] = dados_nasa_polinomial['Ano(x)']
dados_nasa_polinomial['Y'] = dados_nasa_polinomial['Temperatura_media']

# Aplicando x * y e x^2 para preencher valores no tabelamento
dados_nasa_polinomial['x*y'] = dados_nasa_polinomial[['X', 'Y']].apply(xy, axis=1)
dados_nasa_polinomial['x^2'] = dados_nasa_polinomial['X'].apply(x_quadrado)
dados_nasa_polinomial['x^3'] = dados_nasa_polinomial['X'].apply(lambda x: x * x * x)
dados_nasa_polinomial['x^4'] = dados_nasa_polinomial['X'].apply(lambda x: x * x * x * x)
dados_nasa_polinomial['(x^2)*Y'] = dados_nasa_polinomial[['X', 'Y']].apply(x_quadrado_y, axis=1)
dados_nasa_polinomial.head()

,Ano(x),Temperatura_media,X,Y,x*y,x^2,x^3,x^4,(x^2)*Y,S_Q_REG,S_Q_TOTAL,g(x)
0,1880,13.477273,1880,13.477273,25337.272727,3534400,6644672000,12491983360000,4.763407e+07,None,None,None
1,1890,13.494000,1890,13.494000,25503.660000,3572100,6751269000,12759898410000,4.820192e+07,None,None,None
2,1900,13.354000,1900,13.354000,25372.600000,3610000,6859000000,13032100000000,4.820794e+07,None,None,None
3,1910,13.391000,1910,13.391000,25576.810000,3648100,6967871000,13308633610000,4.885171e+07,None,None,None
4,1920,13.473000,1920,13.473000,25868.160000,3686400,7077888000,13589544960000,4.966687e+07,None,None,None


In [ ]:
# Determinando função polinomial
def polinomial(a, b, c, x):
  return (a * (x*x)) + b * x + c

## Calculo dos coeficientes a, b e c do caso polinomial
<img src="https://i.imgur.com/xfqIbCp.png" width="500">


In [ ]:
# Determinando valor de n baseado no numero de linhas do dataframe
n = dados_nasa_polinomial.shape[0]

# Calculando somatorio dos valores tabelados para aplicar no sistema linear e encontrar a, b e c
somatorio_x = somatorio(dados_nasa_polinomial['X'])
somatorio_y = somatorio(dados_nasa_polinomial['Y'])
somatorio_xy = somatorio(dados_nasa_polinomial['x*y'])
somatorio_x_quadrado = somatorio(dados_nasa_polinomial['x^2'])
somatorio_x_cubo = somatorio(dados_nasa_polinomial['x^3'])
somatorio_x_quarta = somatorio(dados_nasa_polinomial['x^4'])
somatorio_x_ao_quadrado_y = somatorio(dados_nasa_polinomial['(x^2)*Y'])

# Montando sistema linear
# Matriz A
A = np.array([[n, somatorio_x, somatorio_x_quadrado],
              [somatorio_x, somatorio_x_quadrado, somatorio_x_cubo],
              [somatorio_x_quadrado, somatorio_x_cubo, somatorio_x_quarta]], dtype="float64"
             )
B = np.array([somatorio_y, somatorio_xy, somatorio_x_ao_quadrado_y], dtype="float64")

sistema = SistEqLineares(A, B)
sistema_aumentado_triangular_superior = sistema.eliminacao_gauss(sistema, sistema.A, sistema.b)

# Agora, a solução do sistema é trivial e basta aplicar a retrosubstituição
# Calculando coeficiente a
#a = sistema.A[2][2] / sistema.b[2]
a_polinomial = sistema.A_B[2][3] / sistema.A_B[2][2]
b_polinomial = (sistema.A_B[1][3] - sistema.A_B[1][2] * a_polinomial) / sistema.A_B[1][1]
c_polinomial = (sistema.A_B[0][3] - sistema.A_B[0][2] * a_polinomial - sistema.A_B[0][1] * b_polinomial) / sistema.A_B[0][0]

K =  1
┌─────────────────────────────────────────────────────────────────────────────────┐
│              15.00            29250.00         57065500.00              207.10 │
│           29250.00         57065500.00     111386925000.00           404078.17 │
│        57065500.00     111386925000.00  217524007270000.00        788795981.53 │
└─────────────────────────────────────────────────────────────────────────────────┘
K =  2
┌─────────────────────────────────────────────────────────────────────┐
│           15.00         29250.00      57065500.00           207.10 │
│            0.00         28000.00     109200000.00           232.64 │
│            0.00     109200000.00  425921253333.34        910607.31 │
└─────────────────────────────────────────────────────────────────────┘
K =  3
┌─────────────────────────────────────────────────────────┐
│        15.00      29250.00   57065500.00        207.10 │
│         0.00      28000.00  109200000.00        232.64 │
│         0.00          0.0

## Printa resultados

In [ ]:
# Cálculo de y médio
y_medio_polinomial = somatorio(dados_nasa_polinomial['Y']) / n

dados_nasa_polinomial['S_Q_REG'] = dados_nasa_polinomial['X'].apply(lambda x: (((a_polinomial * (x*x)) + (b_polinomial * x) + c_polinomial) - y_medio_polinomial)**2)
dados_nasa_polinomial['S_Q_TOTAL'] = dados_nasa_polinomial['Y'].apply(lambda y: (y - y_medio_polinomial)**2)

SQREG   = somatorio(dados_nasa_polinomial['S_Q_REG'])
SQTOTAL = somatorio(dados_nasa_polinomial['S_Q_TOTAL'])

dados_nasa_polinomial.head()

# Cálculo de y médio
print(f"Soma dos Quadrados da Regressão (SSR): { SQREG }")
print(f"Soma dos Quadrados Total (SST): { SQTOTAL }")
print(f"R2: { SQREG / SQTOTAL }")

Soma dos Quadrados da Regressão (SSR): 2.1981428885063354
Soma dos Quadrados Total (SST): 2.269525584573019
R2: 0.9685473049733813


# Caso geométrico
> A função que representa o caso geométrico é a função:
$$
   ln(y) = a*ln(x) + ln(b)
$$


In [ ]:
# Determinando função geométrica
def geometrico(a, b, x):
  return (x * np.log(x)) + np.log(b)

## Tabelamento inicial e colunas x, y, x * y e x^2

In [ ]:
# Criando dataframe para o caso linear
dados_nasa_geometrico = tabelamento_inicial(dados_nasa)

# Preenchendo X(logaritmo natural do ponto) e Y do caso linear
dados_nasa_geometrico['X'] = dados_nasa_geometrico['Ano(x)']
dados_nasa_geometrico['Y'] = dados_nasa_geometrico['Temperatura_media'].apply(np.log)

# Aplicando x * y e x^2 para preencher valores no tabelamento
dados_nasa_geometrico['x*y'] = dados_nasa_geometrico[['X', 'Y']].apply(xy, axis=1)
dados_nasa_geometrico['x^2'] = dados_nasa_geometrico['X'].apply(x_quadrado)
dados_nasa_geometrico.head()

## Calculo dos coeficientes a e b do caso geométrico
<img src="https://i.imgur.com/nLoQLOJ.png" width="300">
<img src="https://i.imgur.com/oZvQ54X.png" width="300" height="91">


In [ ]:
# Determinando valor de n baseado no numero de linhas do dataframe
n = dados_nasa_geometrico.shape[0]

# Calculando somatorio dos valores tabelados para aplicar na equação a e b
somatorio_x = somatorio(dados_nasa_geometrico['X'])
somatorio_y = somatorio(dados_nasa_geometrico['Y'])
somatorio_xy = somatorio(dados_nasa_geometrico['x*y'])
somatorio_x_quadrado = somatorio(dados_nasa_geometrico['x^2'])
somatorio_x_ao_quadrado = somatorio_x * somatorio_x

# Calculando coeficiente a
a_geometrico = ((n * somatorio_xy) - (somatorio_x * somatorio_y)) / ((n * somatorio_x_quadrado) - (somatorio_x_ao_quadrado))
# Calculando coeficiente b
b_geometrico = ((somatorio_x * somatorio_xy) - (somatorio_y * somatorio_x_quadrado)) / (somatorio_x_ao_quadrado - (n * somatorio_x_quadrado))

print(f'a = {a_geometrico}\nb = {b_geometrico}')

## Tabelamento final e resultados do caso geométrico

In [ ]:
# Cálculo de y médio
y_medio_geometrico = somatorio(dados_nasa_geometrico['Y']) / n

dados_nasa_geometrico['S_Q_REG'] = dados_nasa_geometrico['X'].apply(lambda x: (((a_geometrico * x) + b_geometrico) - y_medio_geometrico)**2)
dados_nasa_geometrico['S_Q_TOTAL'] = dados_nasa_geometrico['Y'].apply(lambda y: (y - y_medio_geometrico)**2)

SQREG   = somatorio(dados_nasa_geometrico['S_Q_REG'])
SQTOTAL = somatorio(dados_nasa_geometrico['S_Q_TOTAL'])

dados_nasa_geometrico.head()

# Cálculo de y médio
print(f"Soma dos Quadrados da Regressão (SSR): { SQREG }")
print(f"Soma dos Quadrados Total (SST): { SQTOTAL }")
print(f"R2: { SQREG / SQTOTAL }")

# Caso potência
> A função que representa o caso potência é a função:
$$
   ln(y) = ln(a) * x + ln(b)
$$


In [ ]:
# Determinando função potência
def potencia(a, b, x):
  return (np.log(a) * x) + np.log(b)

## Tabelamento inicial e colunas x, y, x * y e x^2

In [ ]:
# Criando dataframe para o caso potencia
dados_nasa_potencia = tabelamento_inicial(dados_nasa)

In [ ]:
# Preenchendo X(logaritmo natural do ponto) e Y do caso linear
dados_nasa_potencia['X'] = dados_nasa_potencia['Ano(x)'].apply(np.log)
dados_nasa_potencia['Y'] = dados_nasa_potencia['Temperatura_media'].apply(np.log)

# Aplicando x * y e x^2 para preencher valores no tabelamento
dados_nasa_potencia['x*y'] = dados_nasa_potencia[['X', 'Y']].apply(xy, axis=1)
dados_nasa_potencia['x^2'] = dados_nasa_potencia['X'].apply(x_quadrado)
dados_nasa_potencia.head()

## Calculo dos coeficientes a e b do caso potência
<img src="https://i.imgur.com/nLoQLOJ.png" width="300">
<img src="https://i.imgur.com/oZvQ54X.png" width="300" height="91">


In [ ]:
# Determinando valor de n baseado no numero de linhas do dataframe
n = dados_nasa_potencia.shape[0]

# Calculando somatorio dos valores tabelados para aplicar na equação a e b
somatorio_x             = somatorio(dados_nasa_potencia['X'])
somatorio_y             = somatorio(dados_nasa_potencia['Y'])
somatorio_xy            = somatorio(dados_nasa_potencia['x*y'])
somatorio_x_quadrado    = somatorio(dados_nasa_potencia['x^2'])
somatorio_x_ao_quadrado = somatorio_x * somatorio_x

# Calculando coeficiente a
a_potencia = ((n * somatorio_xy) - (somatorio_x * somatorio_y)) / ((n * somatorio_x_quadrado) - (somatorio_x_ao_quadrado))
# Calculando coeficiente b
b_potencia = ((somatorio_x * somatorio_xy) - (somatorio_y * somatorio_x_quadrado)) / (somatorio_x_ao_quadrado - (n * somatorio_x_quadrado))

print(f'a = {a_potencia}\nb = {b_potencia}')

## Tabelamento final e resultados do caso potência

In [ ]:
# Cálculo de y médio
y_medio_potencia = somatorio(dados_nasa_potencia['Y']) / n

dados_nasa_potencia['S_Q_REG'] = dados_nasa_potencia['X'].apply(lambda x: (((a_potencia * x) + b_potencia) - y_medio_potencia)**2)
dados_nasa_potencia['S_Q_TOTAL'] = dados_nasa_potencia['Y'].apply(lambda y: (y - y_medio_potencia)**2)

SQREG   = somatorio(dados_nasa_potencia['S_Q_REG'])
SQTOTAL = somatorio(dados_nasa_potencia['S_Q_TOTAL'])

print(f"Soma dos Quadrados da Regressão (SSR): { SQREG }")
print(f"Soma dos Quadrados Total (SST): { SQTOTAL }")
print(f"R2: { SQREG / SQTOTAL }")
dados_nasa_potencia.head()

In [ ]:
dados_nasa_rna = tabelamento_inicial(dados_nasa)

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

X = np.array(dados_nasa_rna['Ano(x)']).reshape(-1, 1)
y = dados_nasa_rna['Temperatura_media']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=1)
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
predictions = regr.predict(X_test[:2])
score = regr.score(X_test, y_test)
# Assuming you have a list of new years in 'new_years'
new_years = [2050, 2060, 2070]

# Reshape the new years into a 2D array
new_years_array = np.array(new_years).reshape(-1, 1)

# Use the trained model to make predictions
predictions = regr.predict(new_years_array)

# Print the predictions
for year, prediction in zip(new_years, predictions):
    print(f"Year: {year}, Predicted Temperature: {prediction}")

Year: 2050, Predicted Temperature: -41.143109842195656
Year: 2060, Predicted Temperature: -41.3448199208658
Year: 2070, Predicted Temperature: -41.546529999536006
